# Homework

## SQL Avanzado II: Procedimientos y estructuras avanzadas (V3)

**Description:** Crear un trigger que registre en auditoria_eliminaciones los datos de un estudiante al ser eliminado.

**Documentation:**  
This SQL code creates a trigger named `delete_user_trigger` that is executed before a row is deleted from the `estudiantes` table.
The trigger inserts a record into the `auditoria_eliminaciones` table, logging the user ID and the timestamp of the deletion.
It then deletes the row from the `estudiantes` table.
The trigger ensures that deletions are logged for auditing purposes.
 Note: The `auditoria_eliminaciones` table must exist with appropriate columns (`user_id`, `deleted_at`) for this trigger to work.


In [2]:
DROP TRIGGER IF EXISTS delete_user_trigger;

CREATE TRIGGER delete_user_trigger
BEFORE DELETE ON estudiantes
FOR EACH ROW
BEGIN
    INSERT INTO auditoria_eliminaciones (estudiante_id, email, eliminado_en)
    VALUES (OLD.id, OLD.email, NOW());
END


SyntaxError: invalid syntax (3971611215.py, line 1)

In [3]:
import sql_utils

sql_utils.run_non_select_query(
    """
CREATE TRIGGER delete_user_trigger
BEFORE DELETE ON estudiantes
FOR EACH ROW
BEGIN
    INSERT INTO auditoria_eliminaciones (estudiante_id, email, eliminado_en)
    VALUES (OLD.id, OLD.email, NOW());
END;
    """
)


cerrando conexión, sin errores en la consulta


MySQLInterfaceError: Commands out of sync; you can't run this command now

## Auditoría de Eliminaciones: Antes y Después

A continuación se muestra una comparación visual del contenido de la tabla `estudiantes` antes y después de ejecutar una eliminación, junto con el registro correspondiente en la tabla de auditoría.


In [4]:
import sql_utils

id_to_delete = 1

query_select_all = """SELECT * FROM estudiantes;"""
query_to_delete=f"""DELETE FROM estudiantes WHERE id = {id_to_delete};"""
query_auditoria ="""SELECT * FROM auditoria_eliminaciones;"""


print(f'Consulta antes de borrar:\n')

print(sql_utils.run_query(query_select_all))

print(f'\n\n Borrando el registro con id 1... \n\n\n')
sql_utils.run_non_select_query(query_to_delete)

print(f'\n\nConsulta después de borrar:  \n\n')
print(sql_utils.run_query(query_select_all))

print(f' \n\n\nCRegistro de auditoria:  \n\n\n')
print(sql_utils.run_query(query_auditoria))

Consulta antes de borrar:

   id           nombre              email    estado
0   1  SofÃ­a RamÃ­rez   sofia@correo.com    activo
1   2   MartÃ­n GÃ³mez  martin@correo.com    activo
2   3     Elena Torres   elena@correo.com  inactivo


 Borrando el registro con id 1... 



cerrando conexión, sin errores en la consulta


Consulta después de borrar:  


   id           nombre              email    estado
0   1  SofÃ­a RamÃ­rez   sofia@correo.com    activo
1   2   MartÃ­n GÃ³mez  martin@correo.com    activo
2   3     Elena Torres   elena@correo.com  inactivo
 


CRegistro de auditoria:  



Empty DataFrame
Columns: [id, estudiante_id, email, eliminado_en]
Index: []


# Homework 1

## Triggers
**Description:** Crear un trigger que actualice el estado académico del estudiante al insertar una calificación.

**Documentation:**  
El trigger cambia el estado a activo para cualquier estudiante al insertar calificaciones 


In [ ]:
import sql_utils

sql_utils.run_non_select_query(
    """
    CREATE TRIGGER update_status_estudiante
    AFTER INSERT ON calificaciones
    FOR EACH ROW
    BEGIN
        -- Actualizar el estado del estudiante a 'activo' si tiene calificaciones
        UPDATE estudiantes
        SET estado = 'activo'
        WHERE id = NEW.estudiante_id;
    END;
    """
)

Prueba agregar un nuevo estufiante en estado inactivo, agregarle calificacions y hacer select a estudiantes para ver si el trrigger funca 

In [ ]:
import sql_utils

sql_utils.run_query(
    """
    SELECT *
    FROM estudiantes
    """
)

In [ ]:
sql_utils.run_non_select_query("""
INSERT INTO estudiantes (nombre, email, estado)
VALUES ('Juan Perez','jp@mail.com', 'inactivo')
""")

In [ ]:
import sql_utils

sql_utils.run_non_select_query(
    """
    INSERT INTO calificaciones (estudiante_id, materia_id, nota, fecha)
    VALUES (4, 1, 8, '2023-10-01')
    """
)

In [ ]:
import sql_utils

sql_utils.run_query(
    """
    SELECT
        *
    FROM
        estudiantes
    """
)

## Homework 2

## Procedimientos Almacenados

**Description:** Desarrollar un procedimiento para registrar calificaciones, validando que la nota esté entre 0 y 10.

**Documentation:**  
{sql_doc_1}


In [ ]:
import sql_utils
sql_utils.run_non_select_query(
    """
    CREATE PROCEDURE insertar_calificacion(
        IN estudiante_id INT,
        IN materia_id INT,
        IN calificacion DECIMAL(4,2),
        IN fecha DATE
    )
    BEGIN
        IF calificacion < 0 OR calificacion > 10 THEN
            INSERT INTO calificaciones (estudiante_id, materia_id, nota, fecha)
            VALUES (estudiante_id, materia_id, calificacion, fecha);
        ELSE
            -- Lanzar un error si la calificación no es válida
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'La calificación debe estar entre 0 y 10';
        END IF;
    END;
    """
)


In [ ]:
import sql_utils
sql_utils.run_non_select_query(
    """
    CALL insertar_calificacion(4, 1, 19.5, '2023-10-02');
    """
)

## Homework 2

## Procedimientos almacenados
**Description:** Crear un procedimiento para inscribir estudiantes en materias, verificando su estado activo.

**Documentation:**  
{sql_doc_1}


In [ ]:
import sql_utils
sql_utils.run_non_select_query(
    """
    CREATE PROCEDURE inscribir_estudiante(
        IN nombre VARCHAR(100),
        IN email VARCHAR(100),
        IN estado VARCHAR(20)
    )
    BEGIN
        IF estado NOT IN ('activo') THEN
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'El estado del estudiante debe ser "activo"';
        END IF;
        INSERT INTO estudiantes (nombre, email, estado)
        VALUES (nombre, email, estado);
    END;
    """
)

In [ ]:
# Llamar al procedimiento para insertar una calificación
import sql_utils
sql_utils.run_non_select_query(
    """
    CALL inscribir_estudiante('Ana Gomez', 'anita@mail.com', 'activo');
    """
)

In [ ]:
# Llamar al procedimiento para insertar una calificación
import sql_utils
sql_utils.run_query(
    """
    SELECT * FROM estudiantes WHERE email = 'anita@mail.com'
    """
)

### Homework 3

## Funciones
**Description:** Función para calcular el promedio de un estudiante.

**Documentation:**  
{sql_doc_1}


In [100]:
import sql_utils

sql_utils.run_non_select_query(
    """    
    CREATE FUNCTION calcular_promedio_fn(id INT)
    RETURNS DECIMAL(4,2)
    DETERMINISTIC
    BEGIN
        DECLARE promedio DECIMAL(4,2);
        SELECT AVG(nota) INTO promedio
        FROM calificaciones
        WHERE id = estudiante_id;
        RETURN promedio;
    END
    """
)

cerrando conexión


In [101]:
# Llamar Función para calcular el promedio de un estudiante.
import sql_utils
sql_utils.run_query(
    """
    SELECT calcular_promedio_fn(4) AS promedio
    """
)

,promedio
0,7.63


### Homework 3

## Funciones
**Description:** Función para determinar el estado académico (ej: "Aprobado" si promedio ≥ 6).

**Documentation:**  
{sql_doc_1}


In [ ]:
import sql_utils
sql_utils.run_non_select_query(
    """
    CREATE FUNCTION validar_estado_academico_fn(estudiante_id INT)
    RETURNS VARCHAR(25)
    DETERMINISTIC
    BEGIN
        DECLARE estado VARCHAR(25);
        DECLARE promedio DECIMAL(4,2);
        SET promedio = calcular_promedio_fn(estudiante_id);
        IF promedio IS NULL THEN
            SET estado = 'Sin calificaciones';
        END IF;
        IF promedio >= 6 THEN
            SET estado = 'Aprobado';
        ELSE
            SET estado = 'Reprobado';
        END IF;
        RETURN estado;
    END;
    """
)

In [ ]:
import sql_utils
sql_utils.run_query(
    """
    SELECT validar_estado_academico_fn(3) AS estado_academico
    """
)

# Homework4

## Vistas
**Description:** Vista que muestre el rendimiento de estudiantes con su promedio y estado.

**Documentation:**  
La vista 'student_performance' muestra el ID, nombre, promedio y estado académico de los estudiantes activos.



In [ ]:
import sql_utils

sql_utils.run_non_select_query(
    """
CREATE OR REPLACE VIEW student_performance AS
SELECT
    e.id AS id,
    e.nombre AS nombre,
    calcular_promedio_fn(e.id) AS promedio,
    validar_estado_academico_fn(e.id) AS estado
FROM
    estudiantes e
WHERE e.estado = 'activo'
"""
)

cerrando conexión


In [ ]:
sql_utils.run_query(
    """
    SELECT * FROM student_performance
    """
)

,id,nombre,promedio,estado
0,3,Elena Torres,4.00,Reprobado
1,4,Juan Perez,7.63,Aprobado
2,5,Ana Gomez,NaN,Reprobado
